In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install folium==0.2.1
!pip install tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip install -q tflite-model-maker
!pip install -q tflite-support

import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [2]:
!unzip -q /content/drive/MyDrive/object_detector/training_set.zip

In [8]:
from pydoc import importfile
classes = importfile("/content/training_set/classes.py")

train_data = object_detector.DataLoader.from_pascal_voc(
    '/content/training_set/train',
    '/content/training_set/train',
    classes.CLASSES
)

val_data = object_detector.DataLoader.from_pascal_voc(
    '/content/training_set/validate',
    '/content/training_set/validate',
    classes.CLASSES
)

In [9]:
spec = model_spec.get('efficientdet_lite0')
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

In [10]:
model.evaluate(val_data)

In [11]:
model.export(export_dir='.', tflite_filename='detector.tflite')

In [12]:
model.evaluate_tflite('detector.tflite', val_data)

In [13]:
!mv detector.tflite /content/drive/MyDrive/object_detector/